In [5]:
import pandas as pd
import numpy as np

In [6]:
TRAINING_DATA_FILE = 'SpamData\02_Training\train-data.txt'
TEST_DATA_FILE = 'SpamData\02_Training\test-data.txt'
VOCAB_SIZE = 2500
# TOKEN_SPAM_PROB_FILE = ' save this file in a path' 
# TOKEN_HAM_PROB_FILE = 'Save the file'
# TOKEN_ALL_PROB_FILE = 'Save the file'
Test_Features_Matrix = 'SpamData\03_Testing\test-features.txt'
Test_target_FIle = 'SpamData\03_Testing\prob-all-tokens.txt'

In [ ]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter= ' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [ ]:
sparse_train_data[ : 5] # 4 categories in this table DOC_ID, Word_ID, Category, OCCURENCE
sparse_test_data[-5:]

In [ ]:
print('No of rows in training file', sparse_train_data.shape[0])
print('No of rows in test file', sparse_test_data.shape[0])

How to Create an Empty DataFrame

In [ ]:
column_names = ['DOC_ID']+['CATEGORY'] + list(range(0, VOCAB_SIZE)) # creating the column name items with categories


In [ ]:
index_names = np.unique(sparse_train_data[:, 0]) # selecting all rows with one column of data
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

Create full matrix from sparse matrix

In [ ]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx = 1, categ_idx = 2, freq_idx = 3):

       column_names = ['DOC_ID']+['CATEGORY'] + list(range(0, VOCAB_SIZE)) # creating the column name items with categories
       doc_id_names = np.unique(sparse_train_data[:, 0])
       full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
       full_matrix.fillna(value=0, inplace=True)

       for i in range(sparse_matrix.shape[0]):
              doc_nr = sparse_matrix[i][0]  # doc_idx denotes 1st column
              word_id = sparse_matrix[i][1]  # word_idx denoted as 2 column here
              label = sparse_matrix[i][2] # Likewise
              occurence = sparse_matrix[i][freq_idx]

              full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
              full_matrix.at[doc_nr, 'CATEGORY'] = label
              full_matrix.at['doc_nr', 'word_id'] = occurence
              full_matrix.set_index['DOC_ID']
       
       return full_matrix


# Training Naives Bayes Model

Calculating the probablity of Spam | Viagra, substitue into the formula

In [ ]:
full_train_data.CATEGORY.size
full_train_data.CATEGORY.sum() # to get the category of spam messages
prob_spam = full_train_data.full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probablity of spam is', prob_spam)

Total no.of words / tokens

In [ ]:
full_train_features = full_train_data.loc[:, full_train_data.columns!= 'CATEGORY'] # selecting the columns 
#with word_id not the CATEGORY column
full_train_features.head()
# to get the total no.of words we have to sum each row in the table
email_lengths = full_train_features.sum(axis=1) # sum up the columns
TOTal_wordcount = email_lengths.sum()

create a subset of email_lengths, that only contains the spam messg and then count tot no.of words in spam email

In [ ]:
spam_length= email_lengths[full_train_data.CATEGORY == 1]
spam_wc = spam_length.sum()

In [ ]:
ham_length= email_lengths[full_train_data.CATEGORY == 0]
ham_length.sum()
non_spam_wordcount = ham_length.sum()

In [ ]:
print('Average nr of words in spam mails'.format(spam_wc/spam_length.shape[0]))
print('Average nr of words in ham mails'.format(non_spam_wordcount/ham_length.shape[0]))

# Summing the Tokens Occuring in Spam

In [ ]:
full_train_features.shape

In [ ]:
#full_train_data is with all columns including category column , full train features is without category of spam classification.
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1 ]
train_spam_tokens.head() # spam hs 1259 so the overall shape should be 9(1249, 2500)

summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # while adding the no.of.
#occurences if one of the row is zero , to make it as non-zero.

# REPEAT the same sprocedure for summing the tokens in HAM


# P( Token | SPam) - Probablity that a token occurs given the email is Spam

In [ ]:
prob_tokens_spam = summed_spam_tokens/(spam_wc + VOCAB_SIZE)
prob_tokens_spam.sum() #--> this is P(Token | Spam)


 P( Token | SPam) - Probablity that a token occurs given the email is Spam

In [ ]:
prob_tokens_nonspam = summed_ham_tokens/(non_spam_wordcount + VOCAB_SIZE) # not defined for summed_ham_tokens
prob_tokens_nonspam.sum()

P(Token) - Probablity that token occurs

In [ ]:
prob_tokens_all = full_train_features.sum(axis=0) / TOTal_wordcount
prob_tokens_all.sum()

Save the trained Model

In [ ]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam) # ( path for file, objects created to use)


# Prepare Test Data

create full matrix from sparse test data with that func created, seperate feature and target values, save the files as test target file and test feature matrix file.

In [ ]:
full_test_data = make_full_matrix(sparse_test_data, nr_words= VOCAB_SIZE)
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY'] # feature
y_test = full_test_data.CATEGORY  # target


In [ ]:
np.savetxt(Test_target_FIle, y_test)
np.savetxt(Test_target_FIle, x_test)

Calcualting joint probablity using dot product

In [11]:
a=np.array([1,2,3])
b=np.array([4,5,6])
a.dot(b)

32

In [ ]:
x_test.dot(prob_tokens_spam)

# Set the prior-- it is guess or belief about some quantity
$$ P(Spam \, | \, X) = \frac{P(X \, | \, Spam  \,) \, P(Spam)} {P(X)} $$

In [ ]:
PROB_SPAM = 0.3116
# taking log for calculation for very small values, taking log will spread the 
#values for plotting in graph, easy to intrepret

# Joint Probability in log format

In [ ]:
joint_log_spam = x_test.dot(np.log(prob_tokens_spam) - np.log(prob_tokens_all)) + np.log(prob_tokens_nonspam)

$$ P(ham \, | \, X) = \frac{P(X \, | \, ham  \,) \, P(ham)} {P(X)} $$

In [ ]:
joint_log_ham = x_test.dot(np.log(prob_tokens_nonspam) - np.log(prob_tokens_all)) + np.log(prob_tokens_nonspam)

# Making Predictions 
Checking for higher joint probability
$$P( spam\, | \, x) > P( ham \, | x \,) $$

In [ ]:
Prediction = joint_log_spam > joint_log_spam